In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb  # LightGBM
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
warnings.filterwarnings("ignore")


In [4]:
def preprocessing_expected(df):
    df['prob_0'] = 100 - df['rainfall_train.v01']
    df['prob_1'] = df['rainfall_train.v01'] - df['rainfall_train.v02']
    df['prob_2'] = df['rainfall_train.v02'] - df['rainfall_train.v03']
    df['prob_3'] = df['rainfall_train.v03'] - df['rainfall_train.v04']
    df['prob_4'] = df['rainfall_train.v04'] - df['rainfall_train.v05']
    df['prob_5'] = df['rainfall_train.v05'] - df['rainfall_train.v06']
    df['prob_6'] = df['rainfall_train.v06'] - df['rainfall_train.v07']
    df['prob_7'] = df['rainfall_train.v07'] - df['rainfall_train.v08']
    df['prob_8'] = df['rainfall_train.v08'] - df['rainfall_train.v09']
    df['prob_9'] = df['rainfall_train.v09']
    df.drop(columns=['rainfall_train.v01',
                    'rainfall_train.v02',
                    'rainfall_train.v03',
                    'rainfall_train.v04',
                    'rainfall_train.v05',
                    'rainfall_train.v06',
                    'rainfall_train.v07',
                    'rainfall_train.v08',
                    'rainfall_train.v09'], inplace=True)
    df = df[df['rainfall_train.vv'] >= 0]
    df["expected"] = (df["prob_0"] * 0\
                    +df["prob_1"] * 1\
                    +df["prob_2"] * 2\
                    +df["prob_3"] * 3\
                    +df["prob_4"] * 4\
                    +df["prob_5"] * 5\
                    +df["prob_6"] * 6\
                    +df["prob_7"] * 7\
                    +df["prob_8"] * 8\
                    +df["prob_9"] * 9) / 100
    df.drop(columns=["prob_0","prob_1","prob_2","prob_3","prob_4","prob_5","prob_6","prob_7","prob_8","prob_9",
                    'Unnamed: 0', 'rainfall_train.fc_year',	'rainfall_train.fc_month',	'rainfall_train.fc_day',	'rainfall_train.fc_hour'
                    ], inplace=True)
    return df

In [6]:
def preprocessing_pivoting(df):
    df["index"] = df["rainfall_train.dh"] // 12
    df["index"].value_counts()
    pivot_df = df.pivot_table(index=['rainfall_train.stn4contest', 
                                    'rainfall_train.ef_year', 
                                    'rainfall_train.ef_month', 
                                    'rainfall_train.ef_day', 
                                    'rainfall_train.ef_hour',
                                    'rainfall_train.vv',
                                    'rainfall_train.class_interval'
                                    ],
                            columns='index',
                            values='expected').reset_index()
    final_df = pivot_df.groupby([
    'rainfall_train.stn4contest', 
    'rainfall_train.ef_year', 
    'rainfall_train.ef_month', 
    'rainfall_train.ef_day', 
    'rainfall_train.ef_hour']).sum().reset_index()
    return final_df


In [11]:
df = pd.read_csv("data/rainfall_train.csv", encoding='utf-8', index_col=False)
df


,Unnamed: 0,rainfall_train.fc_year,rainfall_train.fc_month,rainfall_train.fc_day,rainfall_train.fc_hour,rainfall_train.stn4contest,rainfall_train.dh,rainfall_train.ef_year,rainfall_train.ef_month,rainfall_train.ef_day,...,rainfall_train.v02,rainfall_train.v03,rainfall_train.v04,rainfall_train.v05,rainfall_train.v06,rainfall_train.v07,rainfall_train.v08,rainfall_train.v09,rainfall_train.vv,rainfall_train.class_interval
0,1,A,5,1,9,STN001,3,A,5,1,...,0,0,0,0,0,0,0,0,0.0,0
1,2,A,5,1,9,STN001,6,A,5,1,...,0,0,0,0,0,0,0,0,0.0,0
2,3,A,5,1,9,STN001,9,A,5,1,...,0,0,0,0,0,0,0,0,0.0,0
3,4,A,5,1,9,STN001,12,A,5,1,...,0,0,0,0,0,0,0,0,0.0,0
4,5,A,5,1,9,STN001,15,A,5,2,...,0,0,0,0,0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457247,1457248,C,9,30,21,STN020,228,C,10,10,...,7,2,2,0,0,0,0,0,0.5,3
1457248,1457249,C,9,30,21,STN020,231,C,10,10,...,9,5,3,1,0,0,0,0,0.0,0
1457249,1457250,C,9,30,21,STN020,234,C,10,10,...,9,5,3,1,0,0,0,0,1.9,4
1457250,1457251,C,9,30,21,STN020,237,C,10,10,...,6,4,2,2,0,0,0,0,1.9,4


In [9]:
df[(df["rainfall_train.stn4contest"] == 'STN001') &
   	(df["rainfall_train.ef_year"] == 'A') &	
    (df["rainfall_train.ef_month"] == 8) & 
    (df["rainfall_train.ef_day"] == 1) &
    (df["rainfall_train.ef_hour"] == 15)
       ]

,Unnamed: 0,rainfall_train.fc_year,rainfall_train.fc_month,rainfall_train.fc_day,rainfall_train.fc_hour,rainfall_train.stn4contest,rainfall_train.dh,rainfall_train.ef_year,rainfall_train.ef_month,rainfall_train.ef_day,...,rainfall_train.v02,rainfall_train.v03,rainfall_train.v04,rainfall_train.v05,rainfall_train.v06,rainfall_train.v07,rainfall_train.v08,rainfall_train.v09,rainfall_train.vv,rainfall_train.class_interval
13275,13276,A,7,22,21,STN001,234,A,8,1,...,84,65,40,24,7,2,0,0,21.3,8
13351,13352,A,7,23,9,STN001,222,A,8,1,...,80,64,43,20,9,2,0,0,21.3,8
13427,13428,A,7,23,21,STN001,210,A,8,1,...,80,62,43,25,7,4,0,0,21.3,8
13503,13504,A,7,24,9,STN001,198,A,8,1,...,92,86,56,38,15,4,0,0,21.3,8
13579,13580,A,7,24,21,STN001,186,A,8,1,...,98,89,60,39,18,3,0,0,21.3,8
13655,13656,A,7,25,9,STN001,174,A,8,1,...,95,92,75,46,25,7,1,0,21.3,8
13731,13732,A,7,25,21,STN001,162,A,8,1,...,90,83,68,45,29,9,0,0,21.3,8
13807,13808,A,7,26,9,STN001,150,A,8,1,...,88,79,63,36,13,5,0,0,21.3,8
13883,13884,A,7,26,21,STN001,138,A,8,1,...,88,81,62,38,18,2,0,0,21.3,8
13959,13960,A,7,27,9,STN001,126,A,8,1,...,93,82,65,43,15,3,1,0,21.3,8


In [12]:
df = preprocessing_expected(df)
df

,rainfall_train.stn4contest,rainfall_train.dh,rainfall_train.ef_year,rainfall_train.ef_month,rainfall_train.ef_day,rainfall_train.ef_hour,rainfall_train.vv,rainfall_train.class_interval,expected
0,STN001,3,A,5,1,12,0.0,0,0.00
1,STN001,6,A,5,1,15,0.0,0,0.00
2,STN001,9,A,5,1,18,0.0,0,0.00
3,STN001,12,A,5,1,21,0.0,0,0.00
4,STN001,15,A,5,2,0,0.0,0,0.00
...,...,...,...,...,...,...,...,...,...
1457247,STN020,228,C,10,10,9,0.5,3,0.20
1457248,STN020,231,C,10,10,12,0.0,0,0.31
1457249,STN020,234,C,10,10,15,1.9,4,0.31
1457250,STN020,237,C,10,10,18,1.9,4,0.22


In [13]:
df[(df["rainfall_train.stn4contest"] == 'STN001') &
   	(df["rainfall_train.ef_year"] == 'A') &	
    (df["rainfall_train.ef_month"] == 8) & 
    (df["rainfall_train.ef_day"] == 1) &
    (df["rainfall_train.ef_hour"] == 15)
       ]

,rainfall_train.stn4contest,rainfall_train.dh,rainfall_train.ef_year,rainfall_train.ef_month,rainfall_train.ef_day,rainfall_train.ef_hour,rainfall_train.vv,rainfall_train.class_interval,expected
13275,STN001,234,A,8,1,15,21.3,8,3.12
13351,STN001,222,A,8,1,15,21.3,8,3.08
13427,STN001,210,A,8,1,15,21.3,8,3.08
13503,STN001,198,A,8,1,15,21.3,8,3.85
13579,STN001,186,A,8,1,15,21.3,8,4.05
13655,STN001,174,A,8,1,15,21.3,8,4.36
13731,STN001,162,A,8,1,15,21.3,8,4.18
13807,STN001,150,A,8,1,15,21.3,8,3.76
13883,STN001,138,A,8,1,15,21.3,8,3.79
13959,STN001,126,A,8,1,15,21.3,8,3.97


In [15]:

df = preprocessing_pivoting(df)
df

index,rainfall_train.stn4contest,rainfall_train.ef_year,rainfall_train.ef_month,rainfall_train.ef_day,rainfall_train.ef_hour,rainfall_train.vv,rainfall_train.class_interval,0,1,2,...,11,12,13,14,15,16,17,18,19,20
0,STN001,A,5,1,12,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
1,STN001,A,5,1,15,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
2,STN001,A,5,1,18,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
3,STN001,A,5,1,21,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
4,STN001,A,5,2,0,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77518,STN020,C,10,10,9,0.5,3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.20,0.31
77519,STN020,C,10,10,12,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.31,0.00
77520,STN020,C,10,10,15,1.9,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.31,0.00
77521,STN020,C,10,10,18,1.9,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.22,0.00


In [16]:
df[(df["rainfall_train.stn4contest"] == 'STN001') &
   	(df["rainfall_train.ef_year"] == 'A') &	
    (df["rainfall_train.ef_month"] == 8) & 
    (df["rainfall_train.ef_day"] == 1) &
    (df["rainfall_train.ef_hour"] == 15)
       ]

index,rainfall_train.stn4contest,rainfall_train.ef_year,rainfall_train.ef_month,rainfall_train.ef_day,rainfall_train.ef_hour,rainfall_train.vv,rainfall_train.class_interval,0,1,2,...,11,12,13,14,15,16,17,18,19,20
732,STN001,A,8,1,15,21.3,8,5.92,5.49,5.02,...,3.79,3.76,4.18,4.36,4.05,3.85,3.08,3.08,3.12,0.0


In [20]:
def my_train_test_split(df):
    X = df.drop(columns=[
        'rainfall_train.stn4contest',
        'rainfall_train.ef_year',
        'rainfall_train.ef_hour',
        'rainfall_train.ef_day',
        'rainfall_train.vv',
        'rainfall_train.class_interval'
    ])
    display(X.columns)
    y = df[['rainfall_train.class_interval']]

    X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42) 
    return X_train, y_train, X_val, y_val, X_test, y_test

In [40]:
# H, F, M, C 계산 함수
def calculate_csi_components(df):
    H = ((df['estimated_rank'] != 0) & (df['rainfall_train.class_interval'] != 0) & (df['estimated_rank'] == df['rainfall_train.class_interval'])).sum()
    F = ((df['estimated_rank'] != 0) & (df['estimated_rank'] != df['rainfall_train.class_interval'])).sum()
    M = ((df['estimated_rank'] == 0) & (df['rainfall_train.class_interval'] != 0)).sum()
    C = ((df['estimated_rank'] == 0) & (df['rainfall_train.class_interval'] == 0)).sum()
    return H, F, M, C

# CSI 계산 함수
def calculate_csi(H, F, M):
    return H / (H + F + M) if (H + F + M) > 0 else 0


def score(pred, y_test):
    results_df = pd.DataFrame({'estimated_rank': pred, 'rainfall_train.class_interval': np.array(y_test).ravel()})
        # H, F, M, C 계산
    H, F, M, C = calculate_csi_components(results_df)

    # CSI 계산
    csi_value = calculate_csi(H, F, M)
    print(f"CSI: {csi_value}")
    print(f"H: {H}, F: {F}, M: {M}, C: {C}")


In [21]:
X_train, y_train, X_val, y_val, X_test, y_test = my_train_test_split(df)

Index(['rainfall_train.ef_month',                         0,
                               1,                         2,
                               3,                         4,
                               5,                         6,
                               7,                         8,
                               9,                        10,
                              11,                        12,
                              13,                        14,
                              15,                        16,
                              17,                        18,
                              19,                        20],
      dtype='object', name='index')

In [33]:
catboost = CatBoostClassifier(iterations=100, random_seed=42, silent=True)
lgbm = LGBMClassifier(n_estimators=100, random_state=42)
xgb = XGBClassifier(n_estimators=100, random_state=42)

catboost.fit(X_train, y_train)
lgbm.fit(X_train, y_train)
xgb.fit(X_train, y_train)

catboost_val_pred = catboost.predict(X_val)
lgbm_val_pred = lgbm.predict(X_val)
xgb_val_pred = xgb.predict(X_val)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001638 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5362
[LightGBM] [Info] Number of data points in the train set: 46513, number of used features: 22
[LightGBM] [Info] Start training from score -0.167515
[LightGBM] [Info] Start training from score -4.192130
[LightGBM] [Info] Start training from score -3.758152
[LightGBM] [Info] Start training from score -3.822875
[LightGBM] [Info] Start training from score -3.766481
[LightGBM] [Info] Start training from score -3.568942
[LightGBM] [Info] Start training from score -3.990555
[LightGBM] [Info] Start training from score -4.262852
[LightGBM] [Info] Start training from score -5.202310
[LightGBM] [Info] Start training from score -5.395629


In [42]:
pd.Series(catboost_val_pred.flatten()).value_counts()
score(catboost_val_pred.flatten(), y_val)

CSI: 0.11313444401394808
H: 292, F: 995, M: 1294, C: 12924


In [43]:
pd.Series(lgbm_val_pred.flatten()).value_counts()
score(lgbm_val_pred.flatten(), y_val)


CSI: 0.11693861693861694
H: 301, F: 984, M: 1289, C: 12931


In [44]:
pd.Series(xgb_val_pred.flatten()).value_counts()
score(xgb_val_pred.flatten(), y_val)

CSI: 0.12296983758700696
H: 318, F: 1007, M: 1261, C: 12919


In [51]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Assuming y_train, y_val, y_test are in integer form
num_classes = len(np.unique(y_train))

# One-hot encode the labels for LSTM
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_val_cat = to_categorical(y_val, num_classes=num_classes)
y_test_cat = to_categorical(y_test, num_classes=num_classes)

# LSTM을 위한 스케일링
scaler = MinMaxScaler(feature_range=(0, 1))

# 'rainfall_train.ef_month' 컬럼을 제외한 나머지 컬럼 사용
X_train_features = X_train.iloc[:, 1:].values
X_val_features = X_val.iloc[:, 1:].values
X_test_features = X_test.iloc[:, 1:].values

# X_train, X_val, X_test에 대해 정규화 진행
X_train_scaled = scaler.fit_transform(X_train_features)
X_val_scaled = scaler.transform(X_val_features)
X_test_scaled = scaler.transform(X_test_features)

# LSTM 입력 형식으로 변환
timesteps = X_train_scaled.shape[1]

# 3차원 배열로 변환
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, 1))
X_val_lstm = X_val_scaled.reshape((X_val_scaled.shape[0], timesteps, 1))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, 1))

# LSTM 모델 구성
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(timesteps, 1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(num_classes, activation='softmax'))  # 다중 클래스 분류를 위해 softmax 활성화 함수 사용

# Adam 옵티마이저 사용
optimizer = Adam(learning_rate=0.001)

lstm_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
lstm_model.fit(X_train_lstm, y_train_cat, epochs=100, batch_size=32, validation_data=(X_val_lstm, y_val_cat))

# LSTM 예측 (클래스 레이블)
lstm_val_pred_proba = lstm_model.predict(X_val_lstm)
lstm_val_pred = np.argmax(lstm_val_pred_proba, axis=1)

Epoch 1/100
1454/1454 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8402 - loss: 0.7273 - val_accuracy: 0.8494 - val_loss: 0.5520
Epoch 2/100
1454/1454 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8501 - loss: 0.5566 - val_accuracy: 0.8500 - val_loss: 0.5456
Epoch 3/100
1454/1454 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8528 - loss: 0.5446 - val_accuracy: 0.8490 - val_loss: 0.5424
Epoch 4/100
1454/1454 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8532 - loss: 0.5402 - val_accuracy: 0.8531 - val_loss: 0.5301
Epoch 5/100
1454/1454 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8519 - loss: 0.5369 - val_accuracy: 0.8544 - val_loss: 0.5176
Epoch 6/100
1454/1454 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8581 - loss: 0.5115 - val_accuracy: 0.8550 - val_loss: 0.5097
Epoch 7/100
1454/1454 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8545 - loss: 0.5123 - val_accuracy: 0.8542 - val_loss: 0.5057
Epoch 8/100
1454/1454 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8572 - loss: 0

In [67]:
display(pd.Series(lstm_val_pred.flatten()).value_counts())
score(lstm_val_pred.flatten(), y_val)


0    14259
5      541
6      262
7      238
4       83
2       48
3       40
9       17
1        9
8        8
Name: count, dtype: int64

CSI: 0.11265969802555169
H: 291, F: 955, M: 1337, C: 12922


In [64]:
from sklearn.model_selection import KFold

In [70]:
catboost_val_pred = catboost_val_pred.ravel()

In [80]:
# Create a DataFrame with predictions from individual models
stacked_predictions = np.column_stack((catboost_val_pred, lgbm_val_pred, xgb_val_pred, lstm_val_pred))
stacked_df = pd.DataFrame(stacked_predictions, columns=['catboost', 'lgbm', 'xgb', 'lstm'])

# Split the stacked predictions into training and validation sets
X_train_stack, X_val_stack, y_train_stack, y_val_stack = train_test_split(stacked_df, y_val, test_size=0.2, random_state=42)

# Initialize the meta-model (e.g., RandomForestClassifier)
meta_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the meta-model
meta_model.fit(X_train_stack, y_train_stack)

# Predict on the validation set
stacked_val_pred = meta_model.predict(X_val_stack)

In [81]:
pd.Series(stacked_val_pred).value_counts()

3    26
1    26
5    24
9    23
8    20
6    19
4    18
0    15
2    15
7    14
Name: count, dtype: int64

In [75]:
display(pd.Series(stacked_test_pred.flatten()).value_counts())
score(stacked_test_pred.flatten(), y_val)


0    14529
5      617
6      344
7       15
Name: count, dtype: int64

ValueError: All arrays must be of the same length